In [3]:
import pandas as pd

arquivo = "dados_amr.xlsx"
df = pd.read_excel(arquivo)

df.head()



,id_pedido,nome_cliente,modo_envio,estado,vendas,quantidade,desconto,lucro,segmento,região,...,dia_pedido,dia_semana_pedido,mes_pedido,ano_pedido,dia_envio,mes_envio,ano_envio,data_pedido,mes_ano,tempo_preparacao
0,1,Muhammed MacIntyre,Expresso,Paraná,825.174,9,0.30,-117.8820,Corporativo,Sul,...,4,Domingo,9,2016,6,9,2016,42617,42617,2
1,2,Ruben Dartt,Normal,São Paulo,411.332,4,0.15,-4.8392,Consumidor,Sudeste,...,5,Sexta-feira,9,2014,9,9,2014,41887,41887,4
2,3,Liz Pelletier,Mesmo Dia,São Paulo,411.332,4,0.15,-4.8392,Home Office,Sudeste,...,28,Quinta-feira,11,2013,28,11,2013,41606,41606,0
3,4,Liz Pelletier,Expresso,Amapá,241.960,2,0.00,33.8744,Consumidor,Norte,...,30,Segunda-feira,5,2016,31,5,2016,42520,42520,1
4,5,Liz Pelletier,Normal,Sergipe,341.960,2,0.00,78.6508,Home Office,Nordeste,...,31,Terça-feira,12,2013,7,1,2014,41639,41639,7


## Célula 2

Remove duplicadas, verifica dados ausentes e lista os estados únicos.

Isso garante que não haverá informações duplicadas que poderiam distorcer análises e permite identificar problemas de preenchimento.

In [4]:
df = df.drop_duplicates()

print(df.isnull().sum())

print(df['estado'].unique())



id_pedido            0
nome_cliente         0
modo_envio           0
estado               0
vendas               0
quantidade           0
desconto             0
lucro                0
segmento             0
região               0
subcategoria         0
categoria            0
dia_pedido           0
dia_semana_pedido    0
mes_pedido           0
ano_pedido           0
dia_envio            0
mes_envio            0
ano_envio            0
data_pedido          0
mes_ano              0
tempo_preparacao     0
dtype: int64
['Paraná' 'São Paulo' 'Amapá' 'Sergipe' 'Rio de Janeiro' 'Alagoas'
 'Mato Grosso do Sul' 'Santa Catarina' 'Bahia' 'Amazonas' 'Goiás'
 'Minas Gerais' 'Rio Grande do Sul' 'Espírito Santo' 'Pará' 'Maranhão'
 'Ceará' 'Piauí' 'Pernambuco' 'Rio Grande do Norte' 'Roraima'
 'Mato Grosso' 'Distrito Federal' 'Tocantins' 'Paraíba' 'Rondônia' 'Acre']


### Explicação da Célula 3

Junta dia, mês e ano em uma única coluna de data para o pedido e o envio.

Unificar em um único campo facilita filtragens, análises de prazo, e cálculos de tempo entre pedido e envio.

In [13]:
# Criar data_pedido a partir de dia, mês e ano do pedido
df['data_pedido'] = pd.to_datetime({
    'year': df['ano_pedido'],
    'month': df['mes_pedido'],
    'day': df['dia_pedido']
})

df['data_envio'] = pd.to_datetime({
    'year': df['ano_envio'],
    'month': df['mes_envio'],
    'day': df['dia_envio']
})
#remover as anteriores
df = df.drop(columns=['dia_pedido', 'mes_pedido', 'ano_pedido', 'dia_envio', 'mes_envio', 'ano_envio'])

### Explicação da Célula 4

Remove outliers das colunas numéricas com base no intervalo interquartil (IQR), filtrando valores fora do padrão estatístico.

Reduz distorções causadas por valores extremos, melhorando a qualidade dos dados para análises e modelos.

Cria colunas extras par otimização de tempo de trabalho

In [ ]:
print(df.shape)
lista_colunas = df.select_dtypes(include='number').columns.tolist()
for coluna in lista_colunas:
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    df = df[(df[coluna] >= limite_inferior) & (df[coluna] <= limite_superior)]
print(df.shape)

# Margem de lucro
df['margem_lucro'] = df['lucro'] / df['vendas']

# Valor unitário estimado
df['preco_unitario'] = df['vendas'] / df['quantidade']

(4592, 22)
(4420, 22)


### Explicação da Célula 5

Salva a base atualizada em uma nova aba sem perder as abas anteriores.

Cria uma versão salva da base limpa, não perde as versões anteriores e possui base de dados limpa

In [11]:
# Adiciona o DataFrame df como uma nova aba chamada 'Base_Limpa' no arquivo existente 'dados_amr.xlsx'

# Salva as alterações na nova aba sem incluir o índice da tabela

from openpyxl import load_workbook

with pd.ExcelWriter('dados_amr.xlsx', engine='openpyxl', mode='a') as writer:
    df.to_excel(writer, sheet_name='Base_Limpa', index=False)


ValueError: Sheet 'Base_Limpa' already exists and if_sheet_exists is set to 'error'.

### Explicação da Célula 6

Converte e formata as datas para o padrão 'AAAA-MM-DD'.

Garantir um formato único evita erros futuros.


In [ ]:
#conversão para salvamento

print(df['data_pedido'].dtype)
df['data_pedido'] = pd.to_datetime(df['data_pedido'], errors='coerce')
df['data_envio'] = pd.to_datetime(df['data_envio'], errors='coerce')
df['data_pedido'] = df['data_pedido'].dt.strftime('%Y-%m-%d')
df['data_envio'] = df['data_envio'].dt.strftime('%Y-%m-%d')


int64


KeyError: 'data_envio'

### Exportação de pedidos em abas separadas por ano no Excel

O código abaixo foi criado para *gerar automaticamente abas separadas por ano* em um arquivo Excel, organizando os pedidos conforme a data.

O que ele faz:
- *Converte as datas* de pedido e envio para o formato correto (YYYY-MM-DD).
- *Identifica os anos disponíveis* nos dados.
- *Cria uma aba para cada ano*, apenas com os pedidos daquele ano.
- *Remove colunas desnecessárias*, como dia, mês e ano que já foram unificados em uma coluna de data.

Com isso, o arquivo Excel fica *mais organizado e fácil de navegar, com os dados **divididos por ano em abas separadas*, o que facilita análises e consultas futuras.

In [ ]:
df['data_pedido'] = pd.to_datetime({
    'year': df['ano_pedido'],
    'month': df['mes_pedido'],
    'day': df['dia_pedido']
}, errors='coerce')

print(df['data_pedido'].dt.year.unique())

df['data_envio'] = pd.to_datetime({
    'year': df['ano_envio'],
    'month': df['mes_envio'],
    'day': df['dia_envio']
}, errors='coerce')

from openpyxl import load_workbook

# Garantir que os anos estão corretos
anos_disponiveis = sorted(df['data_pedido'].dt.year.unique())

# Criar uma aba para cada ano, com data formatada e sem colunas redundantes
with pd.ExcelWriter('dados_amr.xlsx', engine='openpyxl', mode='a') as writer:
    for ano in anos_disponiveis:
        pedidos_ano = df[df['data_pedido'].dt.year == ano].copy()

        # Formatar as colunas de data apenas como 'YYYY-MM-DD'
        pedidos_ano['data_pedido'] = pedidos_ano['data_pedido'].dt.strftime('%Y-%m-%d')
        pedidos_ano['data_envio'] = pedidos_ano['data_envio'].dt.strftime('%Y-%m-%d')

        # Remover colunas dia, mes e ano
        pedidos_ano = pedidos_ano.drop(columns=[
            'ano_pedido', 'mes_pedido', 'dia_pedido',
            'ano_envio', 'mes_envio', 'dia_envio'
        ], errors='ignore')  # errors='ignore' evita erro caso alguma já tenha sido removida

        # Salvar a aba do ano
        pedidos_ano.to_excel(writer, sheet_name=f'Pedidos_{ano}', index=False)